# Barcode detection in an IMAGE

The general outline of the algorithm is to:
1. Compute the Scharr gradient magnitude representations in both the x and y direction.
2. Subtract the y-gradient from the x-gradient to reveal the barcoded region.
3. Blur and threshold the image.
4. Apply a closing kernel to the thresholded image.
5. Perform a series of dilations and erosions.
6. Find the largest contour in the image, which is now presumably the barcode.

*It is important to note that since this method makes assumptions regarding the gradient
 representations of the image, and thus will only work for horizontal barcodes*

In [1]:
# importing the packages
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
#Defining the read function of file path to acquire the image from the specific filepath location
def cv_imread(filepath):
    cv_img=cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), -1)
    return cv_img


In [3]:
#loading the image 
imagename=input('name of the image along with format: ')
image = cv_imread('C:/Users/neera/Desktop/barcode detection/'+imagename)#Declaring the file path
cv2.imshow("Image", image)
cv2.waitKey(0)

#converting the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Image", gray)
cv2.waitKey(0)

name of the image along with format: test.jpeg


-1

### Image Gradients
OpenCV provides three types of gradient filters or High-pass filters
1. Sobel
2. Schar
3. Laplacian



In [4]:
# compute the Scharr gradient magnitude representation of the images in both the x and y direction
gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
cv2.imshow("Image", gradX)
cv2.waitKey(0)
cv2.imshow("Image", gradY)
cv2.waitKey(0)

-1

In [16]:
# subtract the y-gradient from the x-gradient
gradient = cv2.subtract(gradX, gradY)
cv2.imshow("Image", gradient)
cv2.waitKey(0)
gradient = cv2.convertScaleAbs(gradient)
cv2.imshow("Image", gradient)
cv2.waitKey(0)

-1

In [17]:
# blur and threshold the image
blurred = cv2.blur(gradient, (9, 9))
(_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)
cv2.imshow("Image",thresh)
cv2.waitKey(0)

-1

In [18]:
# construct a closing kernel and apply it to the thresholded image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
cv2.imshow("Image",closed)
cv2.waitKey(0)

-1

### Morphological Operations
A set of operations that process images based on shapes. Morphological operations apply a structuring element to an input image and generate an output image.

*The most basic morphological operations are Erosion and Dilation.* 

They have a wide array of uses, such as
1. Removing noise.
2. Isolation of individual elements and joining disparate elements in an image.
3. Finding of intensity bumps or holes in an image.

#### Dilation
1. This operations consists of convolving an image A with some kernel ( B), which can have any shape or size, usually a square or circle.
2. The kernel B has a defined anchor point, usually being the center of the kernel.
3. As the kernel B is scanned over the image, we compute the maximal pixel value overlapped by B and replace the image pixel in the anchor point position with that maximal value. As you can deduce, this maximizing operation causes bright regions within an image to "grow" (therefore the name dilation).
4. The dilatation operation is: dst(x,y)=max[(x′,y′):element(x′,y′)≠0]*src(x+x′,y+y′).

#### Erosion
1. This operation is the sister of dilation. It computes a local minimum over the area of given kernel.
2. As the kernel B is scanned over the image, we compute the minimal pixel value overlapped by B and replace the image pixel under the anchor point with that minimal value.
3. The erosion operation is: dst(x,y)=min[(x′,y′):element(x′,y′)≠0]*src(x+x′,y+y′).

In [19]:
# perform a series of erosions and dilations
closed = cv2.erode(closed, None, iterations = 4)
cv2.imshow("Image",closed)
cv2.waitKey(0)
closed = cv2.dilate(closed, None, iterations = 4)
cv2.imshow("Image",closed)
cv2.waitKey(0)

-1

In [21]:
# find the contours in the thresholded image, then sort the contours
# by their area, keeping only the largest one
(cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]


In [25]:
# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))


In [33]:
# draw a bounding box arounded the detected barcode and display the
# image
cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
cv2.imshow("Image", image)
cv2.waitKey(0)

-1